In [150]:
from itertools import combinations, product
import numpy as np
import yaml

In [170]:
def operation(x,y):
    """
    Vectorlike merger
    
    Return
    ------
    gcd: greatest common divisor
    result: result of the operation, simplified and sorted from smallest to largest
    """
    op = np.sum(x * y**2) * x - np.sum(x**2 * y) * y
    op = np.sort(op)  ##Ordenando la solución (tanto quiral como "vectorlike")
    gcd = np.gcd.reduce(op)  ##Máximo común divisor de la solución
    
    if np.all((op == 0)): ##Para evitar un arreglo de nan
        result = op
    else:
        result = op/gcd
    return (gcd, result)

In [171]:
def m_value(n):
    """
    m value for both even and odd cases
    """
    if n%2==0:
        m = n/2 -1
    else:
        m = (n-3)/2
    return m

In [173]:
def eq_satisfied(x, gcd):
    """
    This function verifies that equations (z1³+z2³+ ... + zn³ = 0) and (z1+z2+ ... + zn = 0) are 
    satisfied at the same time, and if it is true, then return the input parameters
    
    Parameters
    ----------
    x: it correspond to the solutions z1, z2, ..., zn
    gcd: the greatest common divisor
    
    Return
    ------
    x: it correspond to the solutions z1, z2, ..., zn
    gcd: the greatest common divisor
    """
    eq_3 = np.sum(x**3)
    eq_1 = np.sum(x)
    
    if eq_3 ==0 and eq_1 ==0:
        #print(f'{x} satisfies the equations with gcd = {gcd}')
        return (gcd, x)

In [174]:
eq_satisfied(np.array([-168790., -168790., -151341.,  151360.,  168220.,  169341.]), 1)

In [175]:
def solution(n, test_case = False):
    """
    A list of n random numbers between -30 and 30 is used to construct "l" and "k". 
    This yields a single solution, which can be vectorlike or chiral.
    """
    m = int(m_value(n))
    
    if test_case == True:
        if n == 15: ##Test for odd case
            l = [3, 0, 3, 1, -2, 1]
            k = [-1, 1, 3, 0, -2, 0, -2]
        elif n == 6: ##Test for even case
            l = [1,2]
            k = [1,-2]
    else:
        l = np.random.randint(-30, 30, size=m+1)
        k = np.random.randint(-30, 30, size=m+1)

    
    ##Even case
    if n%2==0:
        l_sub = l[:m]
        k_sub = k[:m]

        v_p = np.append([l[0]], k_sub)
        v_n = np.append([0, 0], l_sub)

        v_p = np.append(v_p, -np.append([l[0]], k_sub))
        v_n = np.append(v_n, -np.array(l_sub))
        z_sol = operation(v_p, v_n)  ##Solution
    
    ##Odd case
    else:
    
        l_sub = l[:m]
        k_sub = k[:m+1]

        u_p = np.append([0], k_sub)
        u_n = np.append(l_sub, [k[0], 0])

        u_p = np.append(u_p, -np.array(k_sub))
        u_n = np.append(u_n, -np.append(l_sub, [k[0]]))
        z_sol = operation(u_p, u_n) ##Solution
        
    #z_sol = eq_satisfied(z_sol[1], z_sol[0])
    return z_sol

# Test for even case

In [176]:
divisor, z_s = solution(6, test_case=True)
z_s

array([-4., -4.,  1.,  1.,  1.,  5.])

# Test for odd case

This result coincides with the article, but with opposite sign 

In [177]:
divisor, z_s = solution(15, test_case=True)
z_s

array([-6., -2., -2., -2., -1., -1., -1., -1., -1., -1.,  3.,  3.,  4.,
        4.,  4.])

# Generalizing

In [227]:
def apply_merger(l, k, n):
    """
    Create the vectorlike for both even and odd cases, and apply the merger operation
    
    Parameters
    ----------
    l: array for construct the vectorlike
    k: array for construct the vectorlike
    n: number of elements in the solution (Z)
    """
    if n%2==0:

        v_p = np.append([l[0]], k)
        v_n = np.append([0, 0], l)

        v_p = np.append(v_p, -np.append([l[0]], k))
        v_n = np.append(v_n, -np.array(l))
        z_sol = operation(v_p, v_n)  ##Solution
    
    ##Odd
    else:

        u_p = np.append([0], k)
        u_n = np.append(l, [k[0], 0])

        u_p = np.append(u_p, -np.array(k))
        u_n = np.append(u_n, -np.append(l, [k[0]]))
        z_sol = operation(u_p, u_n) ##Solution
        
    z_sol = eq_satisfied(z_sol[1], z_sol[0])
    
    return z_sol

In [228]:
##verifying that it works for the test case
test = apply_merger([1,2],[1,-2], 6)
print(f'The test case works, the chiral solution is {test[1]} with a gcd = {test[0]}')

The test case works, the chiral solution is [-4. -4.  1.  1.  1.  5.] with a gcd = 1


In [233]:
def chiral_solution(n):
    m = int(m_value(n))
    
    ##A vectorlike can contain elements between (-30,30)
    sample_list = np.arange(-30,30)
    
    ##Even case
    if n%2==0:
    ##Se eligen "m" valor de la lista que contiene enteros entre (-30,30)
    ##Para esta elección se usa una combinatoria de modo que no se tengan en cuenta casos repetidos, por ejemplo, [1,2] y [2,1]
        ls = np.array(list(combinations(sample_list, m)))
        ks = np.array(list(combinations(sample_list, m)))

    else:
    ##Se eligen "m" y "m+1" valores de la lista que contiene enteros entre (-30,30)
        ls = np.array(list(combinations(sample_list, m)))
        ks = np.array(list(combinations(sample_list, m+1)))

    ##Once the "ls" and "ks" lists (list of lists) have been constructed, with all the 
    ##possible combinations for a specific "m" value, the next step is to create the 
    ##combination between the "ls" and "ks" lists. For this, the indexes or the number 
    ##of lists within the "ls" and "ks" lists are taken into account. These indexes are 
    ##the ones that are now going to be combined using meshgrid
    index_ls = np.arange(len(ls))
    index_ks = np.arange(len(ks))
    mesh = np.array(np.meshgrid(index_ls, index_ks))
    
    ##It is a list of lists. Each list corresponds to an index for "l" and another for "k", 
    ##and contains all possible combinations.
    combs = mesh.T.reshape(-1, 2)
    
    
    ##Para almacenar los z y el valor del máximo común divisor
    z_results = []
    gcd_results = []
    
    for comb in combs: 
        l = ls[comb[0]]
        k = ks[comb[1]]
        
        dummy = apply_merger(l,k,n)
        z_dummy = dummy[1]
        gcd_dummy = dummy[0]
        #Las siguientes líneas verifican si una solución es de la forma "vectorlike", si lo es, la rechaza
        c = 0 ##Contador para "vectorlike" 
        for i in range(1,(len(z_dummy)//2)+1):
        ##La lista con la solución está ordenada, de modo que verificar los elementos en los extremos, y así sucesivamente hacía adentro, 
        ##nos ayuda a identificar si son o no "vectorlike". Si lo son, entonces el contador "c" incrementa. Al final, el que no es
        ##"vectorlike" tiene c=0.
            if z_dummy[i-1]+z_dummy[-i] != 0:
                c += 1
        if c != 0:  #Solo guarda los que no son "vectorlike"
            z_results.append(z_dummy.tolist())
            gcd_results.append(gcd_dummy.tolist())


    return (n, gcd_results, z_results)

# Solution for n = 5

In [234]:
n_5, gcd_5, z_5 = chiral_solution(5)

In [235]:
print(f'For n=5 the number of chiral solutions is {len(z_5)}')

for i in range(5):
    print(f'gcd: {gcd_5[i]}, z: {z_5[i]}')

For n=5 the number of chiral solutions is 95931
gcd: 29, z: [-24388.0, -1682.0, 811.0, 869.0, 24390.0]
gcd: 58, z: [-11367.0, -1218.0, 376.0, 839.0, 11370.0]
gcd: 87, z: [-7046.0, -1044.0, 231.0, 809.0, 7050.0]
gcd: 58, z: [-9800.0, -1885.0, 317.0, 1558.0, 9810.0]
gcd: 145, z: [-3624.0, -870.0, 115.0, 749.0, 3630.0]


# Loading the solutions obtained using multiprocessing

Only works for n=5

In [236]:
with open(r'Chiral_solutions.yaml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    zetas = yaml.full_load(file)

In [6]:
zetas['5']['z_quiral']

[[-24388.0, -1682.0, 811.0, 869.0, 24390.0],
 [-11367.0, -1218.0, 376.0, 839.0, 11370.0],
 [-7046.0, -1044.0, 231.0, 809.0, 7050.0],
 [-9800.0, -1885.0, 317.0, 1558.0, 9810.0],
 [-3624.0, -870.0, 115.0, 749.0, 3630.0],
 [-2783.0, -812.0, 86.0, 719.0, 2790.0],
 [-15334.0, -5336.0, 457.0, 4823.0, 15390.0],
 [-7014.0, -2871.0, 199.0, 2636.0, 7050.0],
 [-4260.0, -2030.0, 113.0, 1887.0, 4290.0],
 [-1159.0, -638.0, 28.0, 599.0, 1170.0],
 [-10458.0, -6612.0, 221.0, 6259.0, 10590.0],
 [-1564.0, -1131.0, 27.0, 1078.0, 1590.0],
 [-8368.0, -6902.0, 103.0, 6617.0, 8550.0],
 [-3705.0, -3480.0, 22.0, 3353.0, 3810.0],
 [-464.0, -434.0, -1.0, 449.0, 450.0],
 [-3451.0, -2834.0, -37.0, 2970.0, 3352.0],
 [-6786.0, -4884.0, -133.0, 5190.0, 6613.0],
 [-1102.0, -693.0, -32.0, 750.0, 1077.0],
 [-6380.0, -3490.0, -251.0, 3870.0, 6251.0],
 [-609.0, -288.0, -31.0, 330.0, 598.0],
 [-1914.0, -776.0, -123.0, 930.0, 1883.0],
 [-2668.0, -917.0, -214.0, 1170.0, 2629.0],
 [-4872.0, -1398.0, -487.0, 1950.0, 4807.0],
 [